In [11]:
import xml.etree.ElementTree as ET
import pandas

In [12]:
tree = ET.parse('data/Thailand_Dives.ssrf')
root = tree.getroot()

In [13]:
root.tag, root.attrib

('divelog', {'program': 'subsurface', 'version': '3'})

In [14]:
divesites = [child.attrib for child in root[1]]
divesites

[{'uuid': '1fb33d15',
  'name': 'Banana Bay - Racha Noi, Thailand',
  'gps': '7.498778 98.324667'},
 {'uuid': '20a60b7f',
  'name': 'Palong Wall - Koh Phi Phi, Thailand',
  'gps': '7.689509 98.765525'},
 {'uuid': '80987779',
  'name': 'Banana Rock - Racha Noi, Thailand',
  'gps': '7.501408 98.327582'},
 {'uuid': '96325c36',
  'name': 'Twins - Koh Tao, Thailand',
  'gps': '9.999084 99.779192'},
 {'uuid': '974ece1b',
  'name': 'Chumpong Rock - Koh Tao, Thailand',
  'gps': '10.161833 99.784639'},
 {'uuid': '9b660b50',
  'name': 'Bay 1 - Racha Yai, Thailand',
  'gps': '7.596972 98.370222'},
 {'uuid': 'ab284c0b',
  'name': 'White Rock - Koh Tao, Thailand',
  'gps': '10.109806 99.813944'},
 {'uuid': 'cd9a8def',
  'name': 'Koh Bida Nok - Koh Phi Phi, Thailand',
  'gps': '7.654055 98.766068'},
 {'uuid': 'd62595ef',
  'name': 'Turtle Rock - Phuket, Thailand',
  'gps': '7.684868 98.763032'},
 {'uuid': 'e6c1cd2b',
  'name': 'Shark Point - Phuket, Thailand',
  'gps': '7.749191 98.576700'},
 {'uuid

In [15]:
dives = []
noDives = 1

# Get all site info
for dive in root[2]:
    # Create dictionary for the dive
    details = {}
    
    # Dive number
    details['no'] = noDives
    
    # Key for the attributes of the dive
    details['details'] = dive.attrib
    
    # Sub element breakdown
    for d in dive:
        # All attributes with text fields
        textField = ['divemaster','buddy','notes','suit']
        
        # Save information based on where it's stored in tree
        if d.tag in textField:
            details[d.tag] = d.text
        else:
            details[d.tag] = d.attrib
    
    # Increase number of dives
    noDives += 1
    
    # Appened details to list
    dives.append(details)

In [16]:
dives

[{'no': 1,
  'details': {'number': '1',
   'rating': '3',
   'visibility': '2',
   'divesiteid': 'cd9a8def',
   'date': '2020-03-10',
   'time': '09:54:00',
   'duration': '51:00 min'},
  'divemaster': 'Tom Domville',
  'buddy': 'Prithvi, Ravi',
  'notes': 'Aussie Divers (PADI), Phuket\nCharter Boat Dive\n\nCurrent: miodre current\nViz: 3-4m\nAvg. Depth: \n\nFish:\nParrot fish, box fish, puffed fish, pipe fish, file fish, blue dragon nuddie, sea cucumbers, spiny lobster, shrimps, clownfish, lionfish, cleaner wrasse, banded snakes, moray eel, damsel fish, yellow back butterfly fish, angelfish, banner fish, moorish idol, crown of thorn sea start, blue star\n\nCoral:\nArella gorgonion, acropora elk horn coral (plate variety at 40ft depth), usmilia, anemone, porites, montipora',
  'suit': '3mm, long wet',
  'cylinder': {'size': '12.0 l',
   'workpressure': '200.0 bar',
   'description': '12ℓ 200 bar',
   'start': '220.0 bar',
   'end': '90.0 bar'},
  'weightsystem': {'weight': '3.6 kg', 'd

In [97]:
needed = {
    'start':'start_pressure',
    'end':'end_pressure',
    'weight':'weight',
    'air': 'air_temp',
    'water': 'water_temp'
}

newDives = []

for dive in dives:
    deets = {}
    
    for key, value in dive.items():

        # How to handle dictionary values
        if type(value) is dict or value is 'details':
            for k, v in value.items():
                deets[k] = deets[v]
                
        # How to handle text values
        else: 
            deets[key] = value
    
    print(deets)
        
    
    print('---------------------------------')
#     newInfo = {}
    
#     for key, value in dive.items():
        
#         if key in subElems['all']:
#             for k, v in value.items():
#                 newInfo[k] = v
                
#         elif key in subElems['part'].keys():
#             for k, v in subElems['part'][key].items():
# #                 print(subElems['part'][key][k],value[k])
# #                 newInfo[subElems['part'][key][k]] = value[k]
#                 print(k,value.keys())
            
#         else:
#             pass

#     newDives.append(newInfo)
#     for key in dive['all']:
#         for k, v in dive[key].items():
#                 newInfo[v] = dive[key][k]
    
#     for key in dive['part'].keys():
#         for k, v in newInfo[key].items():
#                 newInfo[k] = value
    
            
                
#         elif dive[key] is dict:
#             for k, v in newInfo[key].items():
#                 newInfo[k] = value
                
#         else:
#             newInfo[key] = dive[key]
    
#     print(newInfo)

KeyError: '1'

In [58]:
i = 0
for dive in dives:
    for key, value in dive.items():
        try:
            for k, v in value.items():
                if k == 'start':
                    print(i,'this has start!')
        except:
            pass
    i += 1

0 this has start!
1 this has start!
2 this has start!
3 this has start!
4 this has start!
5 this has start!
7 this has start!
8 this has start!
10 this has start!
11 this has start!
12 this has start!
13 this has start!
14 this has start!
15 this has start!


In [ ]:
# Organize Dictionary = 
for dive in dives:
    
    # Unpack 'details' into topline dictionary
    for key, value in dive['details'].items():
        dive[key] = value
    
    # Remove Details Key
    dive.pop('details')

In [ ]:
dives

In [ ]:
newInfo

In [ ]:
def cleanDict(origDict):
    subElems = {
        'cylinder':{
            'start':'start_pressure',
            'end':'end_pressure'
        },
        'weightsystem':{
            'weight':'weight'
        },
        'divetemperature':{
            'air': 'air_temp',
            'water': 'water_temp'
        },
        'divecomputer':{
        }
    }
    
    for key in origDict.keys():
        
        if key in subElems.keys():
            for k, v in subElems[key].items():
                
            for k, v in origDict[key].items():
                origDict[v] = origDict[key][k]
                
        else:
            for k, v in origDict[key].items():
                origDict[k] = value
        
        # Remove Key 
        dive.pop(key)
                

#     if type(resultsDict) is dict:
        
#     else:
#         for k, v in dive[key].items():
#             dive[k] = value

#     # Remove Key 
#     dive.pop(key)
    
#     return dive
    

In [ ]:
for dive in dives:
    cleanDict(dive,dive,dive)

In [ ]:
# Organize Dictionary = 
for dive in dives:
    
    # Unpack 'details' into topline dictionary
    print(type(dive))
    

In [ ]:
# Get all dive info
for child in root[2]:
    print(child.tag, child.attrib)
    for subelem in child:
        print(subelem.tag, subelem.attrib, subelem.text)

In [ ]:
for n in root[2]
    

In [ ]:
print(ET.tostring(root, encoding='utf8').decode('utf8'))

In [ ]:
[elem.tag for elem in root.iter()]

In [ ]:
for child in root:
    for subelem in child:
        print(subelem.attrib)
        for moresubel in subelem:
            print(moresubel.attrib)

In [ ]:
print('\nAll attributes:')
for child in root:
#     for subelem in elem:
#         print(subelem.attrib)
    print(child)

In [ ]:
print('\nAll item data:')
for elem in root:
    print(elem)
    for subelem in elem:
        print(subelem)
        print(subelem.text)